In [32]:
import rasterio as rio
from rasterio.features import dataset_features
import geopandas as gpd
from shapely.geometry.polygon import orient
from shapely.geometry import Polygon
from shapely.ops import unary_union

def rio_to_exterior(fp, simplify = False):
    with rio.open(fp) as src:
        shapes = list(dataset_features(src, bidx=1, as_mask=False, geographic=True, band=False))
        result = gpd.GeoDataFrame.from_dict(shapes, crs = 'EPSG:4326')
        for i in result.index:
            result.iloc[i]['geometry'] = Polygon(result.iloc[i]['geometry']['coordinates'][0])
        boundary = gpd.GeoSeries(unary_union(result['geometry']))
        boundary_gdf = gpd.GeoDataFrame(boundary, columns = ['geometry'], crs = 'EPSG:4326')

    if simplify:
        boundary_gdf = orient(boundary_gdf.simplify(0.01, preserve_topology=False).loc[0],sign=1.0)
    return boundary_gdf

In [33]:
test_boundary = rio_to_exterior('/home/jovyan/isce_sat2/contributors/sam/download/fraser_23306_21020-026_21021-004_0006d_s01_L090HH_01.cor.grd.tiff')

ValueError: Assigning CRS to a GeoDataFrame without a geometry column is not supported. Supply geometry using the 'geometry=' keyword argument, or by providing a DataFrame with column name 'geometry'

In [7]:
#from uavsar_pytools.uavsar_tools import grd_tiff_convert

In [8]:
import rasterio as rio
from rasterio.features import dataset_features
import geopandas as gpd
from shapely.geometry.polygon import orient
from shapely.geometry import Polygon
from shapely.ops import unary_union


In [ ]:

def rio_to_exterior(fp, simplify = False):
    with rio.open(fp) as src:
        shapes = list(dataset_features(src, bidx=1, as_mask=False, geographic=True, band=False))
        result = gpd.GeoDataFrame.from_dict(shapes, crs = 'EPSG:4326')
        for i in result.index:
            result.iloc[i]['geometry'] = Polygon(result.iloc[i]['geometry']['coordinates'][0])
        boundary = gpd.GeoSeries(unary_union(result['geometry']))
        boundary_gdf = gpd.GeoDataFrame(boundary, columns = ['geometry'], crs = 'EPSG:4326')

    if simplify:
        boundary_gdf = orient(boundary_gdf.simplify(0.01, preserve_topology=False).loc[0],sign=1.0)
    return boundary_gdf

In [10]:
fp = '/home/jovyan/isce_sat2/contributors/sam/download/fraser_23306_21020-026_21021-004_0006d_s01_L090HH_01.cor.grd.tiff'


with rio.open(fp) as src:
        shapes = list(dataset_features(src, bidx=1, as_mask=False, geographic=True, band=False))
        print(shapes)
        
        result = gpd.GeoDataFrame.from_dict(shapes, crs = 'EPSG:4326')
        for i in result.index:
            result.iloc[i]['geometry'] = Polygon(result.iloc[i]['geometry']['coordinates'][0])
        boundary = gpd.GeoSeries(unary_union(result['geometry']))
        boundary_gdf = gpd.GeoDataFrame(boundary, columns = ['geometry'], crs = 'EPSG:4326')


[{'type': 'Feature', 'id': 'fraser_23306_21020-026_21021-004_0006d_s01_L090HH_01.cor.grd.tiff:0', 'properties': {'val': 255.0, 'filename': 'fraser_23306_21020-026_21021-004_0006d_s01_L090HH_01.cor.grd.tiff'}, 'bbox': [-106.12287804, 39.65006064, -105.45032424, 40.16465736], 'geometry': {'type': 'Polygon', 'coordinates': [[(-106.12287804, 40.16465736), (-106.12287804, 39.65006064), (-105.45032424, 39.65006064), (-105.45032424, 40.16465736), (-106.12287804, 40.16465736)]]}}]


ValueError: Assigning CRS to a GeoDataFrame without a geometry column is not supported. Supply geometry using the 'geometry=' keyword argument, or by providing a DataFrame with column name 'geometry'

In [13]:
with rio.open(fp) as src:
    shapes = list(dataset_features(src, bidx=1, as_mask=False, geographic=True, band=False))

[{'type': 'Feature',
  'id': 'fraser_23306_21020-026_21021-004_0006d_s01_L090HH_01.cor.grd.tiff:0',
  'properties': {'val': 255.0,
   'filename': 'fraser_23306_21020-026_21021-004_0006d_s01_L090HH_01.cor.grd.tiff'},
  'bbox': [-106.12287804, 39.65006064, -105.45032424, 40.16465736],
  'geometry': {'type': 'Polygon',
   'coordinates': [[(-106.12287804, 40.16465736),
     (-106.12287804, 39.65006064),
     (-105.45032424, 39.65006064),
     (-105.45032424, 40.16465736),
     (-106.12287804, 40.16465736)]]}}]

In [31]:
shapes[0][0]

KeyError: 0